In [ ]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from tensorflow.keras.regularizers import l2
import os
import pickle
import time
import matplotlib.pyplot as plt
import tensorflow as tf

# Hide GPU from visible devices
#tf.config.set_visible_devices([], 'GPU')

os.environ["OMP_NUM_THREADS"] = '12'
os.environ['TF_ENABLE_ONEDNN_OPTS']='1'
#os.environ['CUDA_VISIBLE_DEVICES']='-1'
#tf.config.threading.set_inter_op_parallelism_threads(3) 
#tf.config.threading.set_intra_op_parallelism_threads(3)
#tf.config.set_soft_device_placement(enabled = True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #to hide tensorflow warnings

tf.config.experimental.list_physical_devices()

In [ ]:
#input data function
def load_input_data(input_file):
    with open(input_file, 'rb') as file:
        dataset = pickle.load(file)
        # For our problem we have positions and energy arrays,
        # which are respectively data and labels in common Machine Learning language
        positions_array = np.array([mol['positions'] for mol in dataset])
        energies_array = np.array([mol['energy'] for mol in dataset])

    return positions_array, energies_array

#model creation
def createmodel(input_class, output_class, base_neurons,batch_size, l2par):
    model = Sequential([
        Dense(base_neurons, activation='relu', kernel_regularizer=l2(l2par)), #input_shape=(input_class,),
        Dense(base_neurons/2., activation='relu', kernel_regularizer=l2(l2par)),
        Dense(base_neurons/4., activation='gelu', kernel_regularizer=l2(l2par)),
        Dense(base_neurons/8., activation='gelu', kernel_regularizer=l2(l2par)),
        Dense(base_neurons/16., activation='relu', kernel_regularizer=l2(l2par)),
        Dense(output_class)
    ])
    return model

#model compilation
def compilemodel(model,opt):
    #optimizer training with type of dataset
    model.compile(
        #rms
        loss='mean_squared_error',
        #adam with LR and decay
        optimizer=opt,
        #metrics to monitor
        metrics=['mean_squared_error', 'mae']
    )

In [ ]:
#define input train and test data directory
#input_file = 'saved_dataset/merged_dataset_rand_5a.pickle'
input_file = 'saved_dataset/merged_randcart5_dataset_5a.pickle'
#Load dataset into data and labels (positions=data, energy=labels)
positions_array, energies_array = load_input_data(input_file)

# Splitting the dataset into training, validation, and test sets
# with a 72-20-8% ratio from initial dataset
train_val_data, test_data, train_val_labels, test_labels = train_test_split(
    positions_array, energies_array, test_size=0.01, random_state=42)
# Further splitting train, into new train and validation samples
# Takes 80% and is divided to 10% of that for the validation set
train_data, val_data, train_labels, val_labels = train_test_split(
    train_val_data, train_val_labels, test_size=0.1, random_state=42)

print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')
print(val_data.shape[0], 'validation samples')
print(train_data.shape[1], 'train shape')
#print(train_data)

In [ ]:
#define input, output dimensions
input_class = train_data.shape[1] #dH1,dH2,theta, in Angstrom units
output_class = 1 #total system energy in eV
n_iterations = 1
#define learing rate, epochs and batch_size for optimizer and fitter
batch_size = 128
epochs = 40000
neurons = 256
l2par=0.001
patience=150

In [ ]:
# Base model creation
#iterations = n_iterations
#increment = (end_neurons- base_neurons) // (iterations-1)
#neurons = base_neurons + i*increment
model = createmodel(input_class, output_class, neurons, batch_size, l2par)
#neurons = neurons*2
    
#model name for saving purposes
ainame = 'Adadelta6_'
n_layers = len(model.layers)
specs = str(n_layers)+'layers_'+str(neurons)+'neurons_'
specs2 = str(epochs)+'epochs_'+str(batch_size)+'batchsize_'+str(l2par)+'l2par'

model_name = str(ainame) + str(specs) + str(specs2)


fitstart = time.time()
# Model compilation with automatic LR by adam and adamax to fine descent
# Model training 
compilemodel(model,keras.optimizers.Adadelta())
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience)
history=model.fit(
    train_data, train_labels, epochs=epochs, batch_size=batch_size, 
    validation_data=(val_data, val_labels), 
    callbacks=callback, verbose = 1
    )

#second part of the training with more relaxed optimizer
history2=0
more_training=0
if more_training==1:
    compilemodel(model,keras.optimizers.RMSprop())
    callback2 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
    history2=model.fit(
        train_data, train_labels, epochs=epochs, batch_size=batch_size, 
        validation_data=(val_data, val_labels), 
        callbacks=callback2, verbose = 1
        )
    
fitend = time.time()
print(f'Time passed for fit: {fitend-fitstart:.1f} seconds')
    
# Save model and weights
#create directory to save weighted model
save_dir = os.path.join(os.getcwd(), 'saved_model')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name + '.keras')
model.save(model_path)
print('Saved trained model at %s ' % model_path , '\n')

In [ ]:
#plotting test and predictions for various scenarios

#setting bigger font to ease the eyes
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

    # Estrazione dell'andamento dell'errore sul set di addestramento e sul set di validazione
if more_training==1:
    train_loss = history.history['loss'] + history2.history['loss']
    val_loss = history.history['val_loss'] + history2.history['val_loss']
else:
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
# Creazione del grafico di errore di training e validazione
plot_dir = os.path.join(os.getcwd(), 'model_plots')
if not os.path.isdir(plot_dir):
    os.makedirs(plot_dir)
plot_name = model_name + '__errplot.png'
plot_path = os.path.join(plot_dir, plot_name)
    
plt.figure(figsize=(15, 10))
plt.plot(train_loss, label='Perdita sul set di addestramento')
plt.plot(val_loss, label='Perdita sul set di validazione')
plt.xlabel('Epoche')
plt.ylabel('Loss function')
plt.ylim(0, 5)
plt.grid(axis='both')
plt.legend()
#plt.title('Andamento dell\'errore')
plt.savefig(plot_path)

    
    #O_HH plot save
input_file = 'saved_dataset/dataset_O_HH_100_5a_test.pickle' 

positions, energies = load_input_data(input_file)
d1 = positions[:,0] #dH1>dH2
d2 = positions[:,1]
theta = positions[:,2]

input_data = positions
tf.function(autograph=False, reduce_retracing=True)
    # Effettua previsioni sulle nuove osservazioni
predictions = model.predict(input_data)

    # Creazione del grafico di O_HH
graph_dir = os.path.join(os.getcwd(), 'model_graphs_O_HH')
if not os.path.isdir(graph_dir):
    os.makedirs(graph_dir)
graph1_name = model_name + '__O_HHfit.png'
graph1_path = os.path.join(graph_dir, graph1_name)
    
fig1, ax1 = plt.subplots(figsize=(21.4,12))
ax1.set(title='Predizione AI per O_HH: 0.1<theta<pi/2 e dH1=dH2= 2A')
ax1.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
ax1.scatter(theta, energies, color ='salmon', label='pw.x energies')
ax1.scatter(theta, predictions, color='darkcyan', label='AI predictions')
ax1.grid(axis='both')
ax1.legend(loc='best', ncol=1, facecolor= 'white')
fig1.savefig(graph1_path, dpi = 200)

    
    #theta plot save
input_file2 = 'saved_dataset/dataset_theta_100_5a_test.pickle' 

positions2, energies2 = load_input_data(input_file2)
d12 = positions2[:,0] #dH1>dH2
d22 = positions2[:,1]
theta2 = positions2[:,2]

input_data2 = positions2
    # Effettua previsioni sulle nuove osservazioni
predictions2 = model.predict(input_data2)

    # Creazione del grafico di theta
graph2_dir = os.path.join(os.getcwd(), 'model_graphs_theta')
if not os.path.isdir(graph2_dir):
    os.makedirs(graph2_dir)
graph2_name = model_name + '__thetafit.png'
graph2_path = os.path.join(graph2_dir, graph2_name)
    
fig2, ax2 = plt.subplots(figsize=(21.4,12))
ax2.set(title='Predizione AI per 1<theta<pi e dH1=dH2= 0.96A')
ax2.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
ax2.scatter(theta2, energies2, color ='salmon', label='pw.x energies')
ax2.scatter(theta2, predictions2, color='darkcyan', label='AI predictions')
ax2.grid(axis='both')
ax2.legend(loc='best', ncol=1, facecolor= 'white')
fig2.savefig(graph2_path, dpi = 200)

    
    #dh2 plot save
input_file3 = 'saved_dataset/dataset_dh2_100_5a_test.pickle' 

positions3, energies3 = load_input_data(input_file3)
d13 = positions3[:,0] #dH1>dH2
d23 = positions3[:,1]
theta3 = positions3[:,2]

input_data3 = positions3
    # Effettua previsioni sulle nuove osservazioni
predictions3 = model.predict(input_data3)

    # Creazione del grafico di theta
graph3_dir = os.path.join(os.getcwd(), 'model_graphs_dh2')
if not os.path.isdir(graph3_dir):
    os.makedirs(graph3_dir)
graph3_name = model_name + '__dh2fit.png'
graph3_path = os.path.join(graph3_dir, graph3_name)
    
fig3, ax3 = plt.subplots(figsize=(21.4,12))
ax3.set(title='Predizione AI per 0.6<dH2<3.6, dH1=0.96A, theta=1.82rad')
ax3.set(xlabel="dH2 [Angstrom]", ylabel="Energy [eV]")
ax3.scatter(d23, energies3, color ='salmon', label='pw.x energies')
ax3.scatter(d23, predictions3, color='darkcyan', label='AI predictions')
ax3.grid(axis='both')
ax3.legend(loc='best', ncol=1, facecolor= 'white')
fig3.savefig(graph3_path, dpi = 200)
    
    #OH_H plot save
input_file4 = 'saved_dataset/dataset_OH_H_100_5a_test.pickle' 

positions4, energies4 = load_input_data(input_file4)
d14 = positions4[:,0] #dH1>dH2
d24 = positions4[:,1]
theta4 = positions4[:,2]

input_data4 = positions4
    # Effettua previsioni sulle nuove osservazioni
predictions4 = model.predict(input_data4)

    # Creazione del grafico di OH_H
graph4_dir = os.path.join(os.getcwd(), 'model_graphs_OH_H')
if not os.path.isdir(graph4_dir):
    os.makedirs(graph4_dir)
graph4_name = model_name + '__OH_Hfit.png'
graph4_path = os.path.join(graph4_dir, graph4_name)
    
fig4, ax4 = plt.subplots(figsize=(21.4,12))
ax4.set(title='Predizione AI per OH_H: 0.6<dH1<2, theta=1.82rad e dH2= 3A')
ax4.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
ax4.scatter(d14, energies4, color ='salmon', label='pw.x energies')
ax4.scatter(d14, predictions4, color='darkcyan', label='AI predictions')
ax4.grid(axis='both')
ax4.legend(loc='best', ncol=1, facecolor= 'white')
fig4.savefig(graph4_path, dpi = 200)  
    
        #OH_H plot save
input_file5 = 'saved_dataset/dataset_O_H_H_100_5a_test.pickle' 

positions5, energies5 = load_input_data(input_file5)
d15 = positions5[:,0] #dH1>dH2
d25 = positions5[:,1]
theta5 = positions5[:,2]

input_data5 = positions5
    # Effettua previsioni sulle nuove osservazioni
predictions5 = model.predict(input_data5)

    # Creazione del grafico di OH_H
graph5_dir = os.path.join(os.getcwd(), 'model_graphs_O_H_H')
if not os.path.isdir(graph5_dir):
    os.makedirs(graph5_dir)
graph5_name = model_name + '__O_H_Hfit.png'
graph5_path = os.path.join(graph5_dir, graph5_name)
    
fig5, ax5 = plt.subplots(figsize=(21.4,12))
ax5.set(title='Predizione AI per O_H_H: 0.5<dH1<3.5 A, theta=0.37 rad e dH2= 2A')
ax5.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
ax5.scatter(d15, energies5, color ='salmon', label='pw.x energies')
ax5.scatter(d15, predictions5, color='darkcyan', label='AI predictions')
ax5.grid(axis='both')
ax5.legend(loc='best', ncol=1, facecolor= 'white')
fig5.savefig(graph5_path, dpi = 200)  
    
plt.close('all')
    
print(model.summary())